In [7]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import json
pylab.rcParams['figure.figsize'] = (8.0, 10.0)


In [8]:
dataDir='/Users/azakaria/Downloads'
#dataType='val2017'
annFile=f'/Users/azakaria/Downloads/nightowls_validation.json'

In [9]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=0.33s)
creating index...
index created!


In [10]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
cat_map = {}
for i,cat in enumerate(cats):
    cat_map[cat['id']] = i
#print(len(cats))
#print(coco.anns.keys()) #keys are the image ids I believe
#print(coco.anns[1768].keys())
nms=[cat['name'] for cat in cats]
#print(nms)
#print([cat['id'] for cat in cats])
#print(cat_map)
#for ann in coco.loadAnns(coco.getAnnIds(imgIds = imgId, catIds=catIds)):
#for ann in coco.loadAnns(coco.getAnnIds()):
    #print(ann['category_id'])

In [11]:
catIds = coco.getCatIds()
print(catIds)
def get_img_ids_for_training(catIds):
    print(catIds)
    imgIds = []
    for catId in catIds:
        imgIds_current_batch = coco.getImgIds(catIds=[catId])[:3000]
        print(imgIds_current_batch)
        if(len(imgIds_current_batch) < 30):
            print('ERROR! Not 30 images in this category!')
        imgIds = imgIds + imgIds_current_batch
    return imgIds

[1, 2, 3, 4]


In [14]:
from pathlib import Path
def coco_to_yolo_bb(x1, y1, w, h, image_w, image_h):
    #"bbox": [x,y,width,height], (COCO)
    #x_center y_center width height (YOLO)
    return [((2*x1 + w)/(2*image_w)) , ((2*y1 + h)/(2*image_h)), w/image_w, h/image_h]
ann_c = set()
def write_label_file(imgId):
    # using with statement
    img = coco.loadImgs(imgId)[0] #If only 1 imgId, len() = 1
    label_dir = Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels')
    filename = img['file_name'].split('.')[0] + '.txt'
    #print(img['file_name'])
    with open(label_dir/filename, 'w') as file:
        for ann in coco.loadAnns(coco.getAnnIds(imgIds = imgId, catIds=catIds)):
            """
            print(len(ann))
            print(f'img: {img}')
            print(f'ann: {ann}')
            print('***********')
            """
            ann_c.add(ann["category_id"])
            x_center, y_center, width, height = coco_to_yolo_bb(*ann['bbox'], img['width'], img['height'])
            file.write(f'{ann["category_id"]}\t{x_center}\t{y_center}\t{width}\t{height}\n')
    return

img_ids = coco.getImgIds()[:30]
for img_id in img_ids:
    write_label_file(img_id)

-----------------
[{'height': 640, 'width': 1024, 'daytime': 'night', 'file_name': '58c5812cbc260137e096966f.png', 'id': 7000000, 'recordings_id': None, 'timestamp': 2692772762}]
-----------------
-----------------
[{'height': 640, 'width': 1024, 'daytime': 'night', 'file_name': '58c5812cbc260137e0969672.png', 'id': 7000001, 'recordings_id': None, 'timestamp': 2692953347}]
-----------------
-----------------
[{'height': 640, 'width': 1024, 'daytime': 'night', 'file_name': '58c5812cbc260137e0969674.png', 'id': 7000002, 'recordings_id': None, 'timestamp': 2693073987}]
-----------------
-----------------
[{'height': 640, 'width': 1024, 'daytime': 'night', 'file_name': '58c5812cbc260137e0969675.png', 'id': 7000003, 'recordings_id': None, 'timestamp': 2693134287}]
-----------------
-----------------
[{'height': 640, 'width': 1024, 'daytime': 'night', 'file_name': '58c5812cbc260137e0969676.png', 'id': 7000004, 'recordings_id': None, 'timestamp': 2693194602}]
-----------------
---------------

In [ ]:
#print(catIds)
#img_ids = get_img_ids_for_training(catIds)
#img_ids = get
#print(catIds)
#img_ids = coco.getImgIds(catIds=catIds)[:30]

In [18]:
import os
import shutil
from pathlib import Path
import os.path

#I might not need to do this
#Maybe I can just get the images and write them to a file
def copy_images_for_dataset_dir(src_image_dir, dest_image_dir, image_extension):
    fs = os.listdir('/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels')
    for f in fs:
        src = str(src_image_dir/f.split('.')[0]) + image_extension
        dest = str(dest_image_dir/ f.split('.')[0]) + image_extension
        print(src)
        print(dest)
        print('----------------')
        if os.path.isfile(src):
            shutil.copyfile(src, dest)
            #print(f'{src} is a file')
        else:
            print(f'{src} is not a file')
    return
copy_images_for_dataset_dir(Path('/Users/azakaria/Downloads/nightowls_validation'),Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/'), '.png')

/Users/azakaria/Downloads/nightowls_validation/58c5812cbc260137e096967f.png
/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/58c5812cbc260137e096967f.png
----------------
/Users/azakaria/Downloads/nightowls_validation/58c5812cbc260137e096967e.png
/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/58c5812cbc260137e096967e.png
----------------
/Users/azakaria/Downloads/nightowls_validation/58c5812cbc260137e0969672.png
/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/58c5812cbc260137e0969672.png
----------------
/Users/azakaria/Downloads/nightowls_validation/58c5812cbc260137e096967d.png
/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/58c5812cbc260137e096967d.png
----------------
/Users/azakaria/Downloads/nightowls_validation/58c5812cbc260137e0969676.png
/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/58c5812cbc260137e0969676.png
----------------
/Users/azakaria/Downloads/nightowls_validation/58c